In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import Prob_models as PM
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import tensorboard
tfk = tf.keras
tfkl = tf.keras.layers
tfpl = tfp.layers
tfd = tfp.distributions
import random


2022-11-18 09:17:35.140549: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
negative_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)
seed = 100
random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

# Radius 

## Radius generation

In [3]:
A = 2*tfd.Uniform().sample(1000)
X = tfd.InverseGamma(concentration =1.5, scale = 0.6 ).sample(1000)
R1 = A*X


random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
train_dataset = R1[:250]
train_dataset = tf.reshape(train_dataset,[250,1])
eval_dataset = R1[250:]
eval_dataset=tf.reshape(eval_dataset,[750,1])
print(np.max(R1))
print(np.max(R1[:250]))

2022-11-18 09:18:15.111302: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-18 09:18:15.114063: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


165.50166
16.883898


In [4]:
Ber = tfd.Bernoulli(probs=0.3, dtype ='float').sample(1000) 
IG1 = tfd.InverseGamma(concentration =1.8, scale = 1 ).sample(1000)
IG2 = tfd.InverseGamma(concentration =1.8, scale = 1 ).sample(1000)
R2 = IG1*Ber +(6+IG2)*(1-Ber)

train_dataset = R2[:250]
train_dataset = tf.reshape(train_dataset,[250,1])
eval_dataset = R2[250:]
eval_dataset=tf.reshape(eval_dataset,[750,1])
print(np.max(R2))
print(np.max(R2[:250]))

41.40624
41.40624


# Import model

Importing standard VAE model : vae = PM.Std_VAE()
Importing Extreme value VAE model assuming known shape parameter: vae = PM.Ext_VAE()
Importing Extreme value VAE model, shape parameter is learned : vae = PM.U_Ext_VAE()

In [4]:
# For R1 : 
#vae = PM.Std_VAE()
vae = PM.Ext_VAE()
#vae  = PM.U_Ext_VAE()

# For R2 :
#vae = PM.Ext_Multi_VAE()

# Training

In [5]:
checkpoint_filepath = '/home/nlafon/These/Extreme_VAE/tmp/radius1/Ext_VAE/Gamma_output7/checkpoint'
metric ='val_loss'
model_checkpoint_callback = tfk.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor=metric,
    mode='min',
    save_best_only=True)

In [6]:
# U_Ext_VAE
enc = vae.encoder
enc.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-4),
            loss=negative_log_likelihood)

enc.fit(x=train_dataset,
	y=train_dataset, 
        #validation_data=(eval_dataset,eval_dataset), 
        batch_size=32,
        epochs=10)
print(enc.prior.concentration)

Epoch 1/10
8/8 [==============================] - 1s 3ms/step - loss: 1.5587
Epoch 2/10
8/8 [==============================] - 0s 3ms/step - loss: 1.5568
Epoch 3/10
8/8 [==============================] - 0s 4ms/step - loss: 1.5547
Epoch 4/10
8/8 [==============================] - 0s 4ms/step - loss: 1.5528
Epoch 5/10
8/8 [==============================] - 0s 4ms/step - loss: 1.5507
Epoch 6/10
8/8 [==============================] - 0s 4ms/step - loss: 1.5488
Epoch 7/10
8/8 [==============================] - 0s 3ms/step - loss: 1.5469
Epoch 8/10
8/8 [==============================] - 0s 6ms/step - loss: 1.5449
Epoch 9/10
8/8 [==============================] - 0s 4ms/step - loss: 1.5430
Epoch 10/10
8/8 [==============================] - 0s 4ms/step - loss: 1.5410
tf.Tensor(1.5, shape=(), dtype=float32)


In [10]:
negative_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)

vae.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3),
            loss=negative_log_likelihood)

In [11]:
vae.fit(train_dataset,train_dataset, 
        validation_data=(eval_dataset,eval_dataset), 
        batch_size=32,
        epochs=1000, 
        callbacks = [model_checkpoint_callback]
       )

Epoch 1/1000
8/8 [==============================] - 2s 80ms/step - loss: 1.3172 - val_loss: 1.4887
Epoch 2/1000
8/8 [==============================] - 0s 38ms/step - loss: 1.3903 - val_loss: 1.4412
Epoch 3/1000
8/8 [==============================] - 0s 29ms/step - loss: 1.3857 - val_loss: 1.5083
Epoch 4/1000
8/8 [==============================] - 0s 26ms/step - loss: 1.3847 - val_loss: 1.4852
Epoch 5/1000
8/8 [==============================] - 0s 31ms/step - loss: 1.3109 - val_loss: 1.5187
Epoch 6/1000
8/8 [==============================] - 0s 32ms/step - loss: 1.2735 - val_loss: 1.5037
Epoch 7/1000
8/8 [==============================] - 0s 28ms/step - loss: 1.3388 - val_loss: 1.4520
Epoch 8/1000
8/8 [==============================] - 0s 31ms/step - loss: 1.3352 - val_loss: 1.5821
Epoch 9/1000
8/8 [==============================] - 0s 29ms/step - loss: 1.3303 - val_loss: 1.4821
Epoch 10/1000
8/8 [==============================] - 0s 47ms/step - loss: 1.2764 - val_loss: 1.4285
Epoch 11/

In [19]:
print(enc.prior.concentration)

<tf.Variable 'prior_c:0' shape=(1,) dtype=float32, numpy=array([1.8404169], dtype=float32)>


# Angle

## Angle generation

Let **X** the multivariate random vector we want to sample from. **X** = **R** **$\Theta$**  
The norm is set to the 1D samples of previous section (namely R1). 
We sample on the multivariate sphere according to a Dirichlet distribution. 
For every radius r,  **$\Theta$** | r is Dirichlet with K=5, $\alpha_1(\mathbf{R}))=\alpha_2(||\mathbf{R}||) = 3\left(2 -min(1, \frac{1}{2||\mathbf{R}||})\right)$, $ \alpha_3=\alpha_4 = \alpha_5=3\left(1 + min(1, \frac{1}{2||\mathbf{R}||})\right)$.   
The latent space of the VAE on the sphere is a multivariate normal distribution with independent components.


In [4]:
def alphas_function(x):
    res = tf.convert_to_tensor([6. -3*tf.minimum(1.,0.5/x), 6. -3*tf.minimum(1.,0.5/x) , 3.+ 3*tf.minimum(1,0.5/x), 3.+ 3*tf.minimum(1,0.5/x), 3.+ 3*tf.minimum(1,0.5/x)])
    return(tf.transpose(res))


alphas          = alphas_function(train_dataset[:,0])
angle_train_dep = tfd.Dirichlet(alphas).sample()
alphas_eval     = alphas_function(eval_dataset[:,0])
angle_eval_dep = tfd.Dirichlet(alphas_eval).sample()

Different setting when **R** = R2. 

## Model training 

In [5]:
checkpoint_filepath = '/home/nlafon/These/Extreme_VAE/tmp/sphere_radius1/diri_output/checkpoint'
metric ='val_loss'
model_checkpoint_callback = tfk.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor=metric,
    mode='min',
    save_best_only=True)

In [9]:
vae = PM.Sphere_VAE()

In [16]:
vae.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-5),
            loss=negative_log_likelihood)

In [17]:
vae.fit([angle_train_dep,train_dataset],angle_train_dep,
        validation_data=([angle_eval_dep,eval_dataset],angle_eval_dep),
        batch_size=32,epochs=100,
       callbacks = [model_checkpoint_callback])

Epoch 1/100
8/8 [==============================] - 2s 90ms/step - loss: -4.8443 - val_loss: -4.8216
Epoch 2/100
8/8 [==============================] - 0s 32ms/step - loss: -4.8347 - val_loss: -4.7859
Epoch 3/100
8/8 [==============================] - 0s 31ms/step - loss: -4.8312 - val_loss: -4.7877
Epoch 4/100
8/8 [==============================] - 0s 31ms/step - loss: -4.8282 - val_loss: -4.7999
Epoch 5/100
8/8 [==============================] - 0s 31ms/step - loss: -4.8372 - val_loss: -4.7612
Epoch 6/100
8/8 [==============================] - 0s 34ms/step - loss: -4.8340 - val_loss: -4.7208
Epoch 7/100
8/8 [==============================] - 0s 32ms/step - loss: -4.8457 - val_loss: -4.7953
Epoch 8/100
8/8 [==============================] - 0s 34ms/step - loss: -4.8514 - val_loss: -4.8056
Epoch 9/100
8/8 [==============================] - 0s 27ms/step - loss: -4.8365 - val_loss: -4.7887
Epoch 10/100
8/8 [==============================] - 0s 27ms/step - loss: -4.8478 - val_loss: -4.7258